In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import scipy
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
from datetime import datetime
import seaborn as sns

In [2]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss

In [3]:
db=pd.read_csv('ml_labelled_Data.csv')

In [4]:
df=db[['Day_of_Week' , 'Weather_Conditions' , 'Road_Surface_Conditions'
                          , 'Light_Conditions','Speed_limit','Junction_Detail','Road_Type','Accident_Severity']]
df.head()

,Day_of_Week,Weather_Conditions,Road_Surface_Conditions,Light_Conditions,Speed_limit,Junction_Detail,Road_Type,Accident_Severity
0,5,6,5,4,30,4,3,1
1,6,2,1,1,30,0,0,2
2,4,2,1,1,30,4,3,2
3,0,2,1,4,30,4,3,2
4,1,2,5,0,30,4,3,2


In [5]:
df2 = df[df['Accident_Severity'] == 2]
df2 = df2.iloc[0:200000,:]
df2['Accident_Severity'].value_counts()
df1 = df[df['Accident_Severity'] == 1]
df0 = df[df['Accident_Severity'] == 0]

In [16]:
df_target=pd.concat([df2,df1,df0])
df_target['Accident_Severity'].value_counts()
df_target.head()

,Day_of_Week,Weather_Conditions,Road_Surface_Conditions,Light_Conditions,Speed_limit,Junction_Detail,Road_Type,Accident_Severity
1,6,2,1,1,30,0,0,2
2,4,2,1,1,30,4,3,2
3,0,2,1,4,30,4,3,2
4,1,2,5,0,30,4,3,2
5,5,6,5,4,30,4,3,2


In [7]:
df.drop(['Accident_Severity'] ,axis = 1)

,Day_of_Week,Weather_Conditions,Road_Surface_Conditions,Light_Conditions,Speed_limit,Junction_Detail,Road_Type
0,5,6,5,4,30,4,3
1,6,2,1,1,30,0,0
2,4,2,1,1,30,4,3
3,0,2,1,4,30,4,3
4,1,2,5,0,30,4,3
...,...,...,...,...,...,...,...
1048570,4,2,1,4,60,4,3
1048571,3,2,3,3,60,4,3
1048572,5,2,3,4,30,9,3
1048573,5,2,5,3,60,9,3


In [8]:
df_final = pd.concat([df,df_target])

In [17]:
#Create independent and Dependent Features
columns = df_target.columns.tolist()
# Filter the columns to remove data we do not want 
columns = [c for c in columns if c not in ["Accident_Severity"]]
# Store the variable we are predicting 
target = "Accident_Severity"
# Define a random state 
state = np.random.RandomState(42)
X = df_target[columns]
Y = df_target[target]
# Print the shapes of X & Y
print(X.shape)
print(Y.shape)

(352692, 7)
(352692,)


In [18]:
# Implementing Oversampling for Handling Imbalanced 
smk = SMOTETomek(random_state=42)
X_res,y_res=smk.fit_resample(X,Y)

In [19]:
import sklearn.model_selection as model_selection
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_res.values, 
                                              y_res.values,test_size=0.20, random_state=99)

In [20]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(Y)))
print('Resampled dataset shape {}'.format(Counter(y_res)))

Original dataset shape Counter({2: 200000, 1: 138192, 0: 14500})
Resampled dataset shape Counter({0: 199982, 1: 199876, 2: 199868})


In [35]:
new_data = pd.concat([X_res,y_res],axis=1)
new_data.head()

,Day_of_Week,Weather_Conditions,Road_Surface_Conditions,Light_Conditions,Speed_limit,Junction_Detail,Road_Type,Accident_Severity
0,6,2,1,1,30,0,0,2
1,4,2,1,1,30,4,3,2
2,0,2,1,4,30,4,3,2
3,1,2,5,0,30,4,3,2
4,5,6,5,4,30,4,3,2


In [37]:
new_data.to_csv('Synthetic_new_data.csv')

In [21]:
## RandomOverSampler to handle imbalanced data

from imblearn.over_sampling import RandomOverSampler

In [22]:
os =  RandomOverSampler()

In [23]:
X_train_res, y_train_res = os.fit_resample(X, Y)

In [ ]:
import sklearn.model_selection as model_selection
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_res.values, 
                                              y_res.values,test_size=0.20, random_state=99)

# Logistic Regression

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
lr = LogisticRegression()
# Fit the model on the trainng data.
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=y_pred)
print("Accuracy", round(accuracy_score(y_pred, y_test)*100,2))
print(sk_report)
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

Accuracy 45.42
              precision    recall  f1-score   support

           0   0.502090  0.599233  0.546377     39896
           1   0.362803  0.164265  0.226141     40246
           2   0.442884  0.602050  0.510345     39804

    accuracy                       0.454221    119946
   macro avg   0.435926  0.455183  0.427621    119946
weighted avg   0.435707  0.454221  0.426969    119946



C:\Users\polis\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Predicted,0,1,2,All
Actual,,,,
0,23907,5688,10301,39896
1,13791,6611,19844,40246
2,9917,5923,23964,39804
All,47615,18222,54109,119946


# Naive bayes

In [26]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
clf = GaussianNB()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=prediction)
print("Accuracy", round(accuracy_score(y_pred, y_test)*100,2))
print(sk_report)
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

Accuracy 45.42
              precision    recall  f1-score   support

           0   0.494574  0.558552  0.524619     39896
           1   0.367325  0.284972  0.320950     40246
           2   0.451060  0.494825  0.471930     39804

    accuracy                       0.445609    119946
   macro avg   0.437653  0.446116  0.439167    119946
weighted avg   0.437438  0.445609  0.438797    119946



Predicted,0,1,2,All
Actual,,,,
0,23907,5688,10301,39896
1,13791,6611,19844,40246
2,9917,5923,23964,39804
All,47615,18222,54109,119946


# Decision Tree

In [27]:
from sklearn.tree import DecisionTreeClassifier

In [28]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree1 = round(decision_tree.score(X_test, y_test) * 100, 2)
sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=Y_pred)
print("Accuracy", acc_decision_tree1)
print(sk_report)
### Confusion Matrix 
pd.crosstab(y_test, Y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

Accuracy 49.61
              precision    recall  f1-score   support

           0   0.547784  0.690621  0.610965     39896
           1   0.425520  0.215351  0.285974     40246
           2   0.472514  0.584991  0.522771     39804

    accuracy                       0.496098    119946
   macro avg   0.481939  0.496988  0.473237    119946
weighted avg   0.481782  0.496098  0.472652    119946



Predicted,0,1,2,All
Actual,,,,
0,27553,4416,7927,39896
1,13512,8667,18067,40246
2,9234,7285,23285,39804
All,50299,20368,49279,119946


# KNN

In [29]:
X_train.shape

(479780, 7)

In [30]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=915)
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=915)

In [31]:
y_pred = neigh.predict(X_test)

C:\Users\polis\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [32]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

prediction = neigh.predict(X_test)
sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=prediction)
print("Accuracy", round(accuracy_score(y_pred, y_test)*100,2))
print(sk_report)
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

C:\Users\polis\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Accuracy 46.84
              precision    recall  f1-score   support

           0   0.498099  0.682901  0.576041     39896
           1   0.390766  0.145952  0.212526     40246
           2   0.459236  0.579364  0.512353     39804

    accuracy                       0.468377    119946
   macro avg   0.449367  0.469406  0.433640    119946
weighted avg   0.449188  0.468377  0.432934    119946



Predicted,0,1,2,All
Actual,,,,
0,27245,3777,8874,39896
1,16091,5874,18281,40246
2,11362,5381,23061,39804
All,54698,15032,50216,119946


# Random Forest

In [34]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=200)
random_forest.fit(X_train,y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_test, y_test)
acc_random_forest1 = round(random_forest.score(X_test, y_test) * 100, 2)

sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=Y_pred)
print("Accuracy" , acc_random_forest1)
print(sk_report)
pd.crosstab(y_test, Y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

Accuracy 49.73
              precision    recall  f1-score   support

           0   0.548847  0.690570  0.611606     39896
           1   0.426586  0.207399  0.279103     40246
           2   0.473207  0.596573  0.527777     39804

    accuracy                       0.497257    119946
   macro avg   0.482880  0.498181  0.472829    119946
weighted avg   0.482723  0.497257  0.472221    119946



Predicted,0,1,2,All
Actual,,,,
0,27551,4322,8023,39896
1,13487,8347,18412,40246
2,9160,6898,23746,39804
All,50198,19567,50181,119946
